In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
rna_tsne = 'RNAseq_tsne_results.tsv'
rna_nmf = 'RNAseq_nmf_results.tsv'
rna_pca = 'RNAseq_pca_results.tsv'
rna_vae = '../../../vae_pancancer/data/encoded_rnaseq_onehidden_warmup_batchnorm.tsv'

copy_pca = 'CopyNumber_pca_results.tsv'
copy_tsne = 'CopyNumber_tsne_results.tsv'
copy_nmf = 'CopyNumber_nmf_results.tsv'

clinical = "vae_encoded_with_clinical.tsv"

In [3]:
rna_tsne_df = pd.read_table(rna_tsne, index_col=0)
rna_nmf_df = pd.read_table(rna_nmf, index_col=0)
rna_pca_df = pd.read_table(rna_pca, index_col=0)
rna_vae_df = pd.read_table(rna_vae, index_col=0)

copy_pca_df = pd.read_table(copy_pca, index_col=0)
copy_tsne_df = pd.read_table(copy_tsne, index_col=0)
copy_nmf_df = pd.read_table(copy_nmf, index_col=0)

clinical_df = pd.read_table(clinical, index_col=0)

In [4]:
clinical_df.head()

,1,2,3,4,5,6,7,8,9,10,...,acronym,disease,organ,drug,ethnicity,percent_tumor_nuclei,gender,sample_type,analysis_center,year_of_diagnosis
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,0.000000,7.870012,0.000000,0.000000,0.071894,1.981403,2.580404,4.991513,0.000000,0.000000,...,GBM,Glioblastoma multiforme,Brain,NaN,not hispanic or latino,90.0,male,Primary Tumor,BI,2005.0
TCGA-02-0055-01,0.397321,4.752499,0.000000,0.000000,0.000000,0.000000,3.468984,0.235408,1.999857,0.172326,...,GBM,Glioblastoma multiforme,Brain,temozolomide,not hispanic or latino,95.0,female,Primary Tumor,BI,2005.0
TCGA-02-2483-01,0.000000,5.565540,0.000000,0.000000,0.801492,1.969184,3.430048,10.232340,0.000000,0.000000,...,GBM,Glioblastoma multiforme,Brain,temodar,not hispanic or latino,80.0,male,Primary Tumor,BI,2008.0
TCGA-02-2485-01,0.000000,8.667418,0.233253,0.000000,0.954965,0.269454,6.222897,4.165497,0.502135,0.000000,...,GBM,Glioblastoma multiforme,Brain,temodar,not hispanic or latino,80.0,male,Primary Tumor,BI,2009.0
TCGA-02-2486-01,0.000000,7.732954,1.151623,0.836602,0.800159,3.017811,4.462164,0.000000,0.523576,0.641049,...,GBM,Glioblastoma multiforme,Brain,temodar,not hispanic or latino,80.0,male,Primary Tumor,BI,2008.0


In [5]:
real_clinical_df = clinical_df.iloc[:, range(101, 117)]
real_clinical_df.index = clinical_df.index

# Fix some clinical variables
real_clinical_df["year_of_diagnosis"] = real_clinical_df["year_of_diagnosis"].fillna(0)
real_clinical_df["year_of_diagnosis"] = real_clinical_df["year_of_diagnosis"].astype(int)
real_clinical_df.loc[(real_clinical_df["year_of_diagnosis"] < 1979), ["year_of_diagnosis"]] = np.nan

# Assign diagnosis year and age to bins
year_interval = pd.IntervalIndex.from_tuples([
        (1978, 2003),
        (2003, 2007),
        (2007, 2009),
        (2009, 2010),
        (2010, 2011),
        (2011, 2013)
    ])

age_interval = pd.IntervalIndex.from_tuples([
        (0, 34),
        (34, 44),
        (44, 50),
        (50, 55),
        (55, 60),
        (60, 65),
        (65, 70),
        (70, 75),
        (75, 95)
    ])

real_clinical_df["age_at_diagnosis"] = pd.cut(real_clinical_df["age_at_diagnosis"], bins = age_interval)
real_clinical_df["year_of_diagnosis"] = pd.cut(real_clinical_df["year_of_diagnosis"], bins = year_interval)

/home/gway/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gway/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gway/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

In [6]:
rna_tsne_df = rna_tsne_df.add_suffix('_rna_tsne')
rna_nmf_df = rna_nmf_df.add_suffix('_rna_nmf')
rna_pca_df = rna_pca_df.add_suffix('_rna_pca')
rna_vae_df = rna_vae_df.add_suffix('_rna_vae')

copy_pca_df = copy_pca_df.add_suffix('_copy_pca')
copy_tsne_df = copy_tsne_df.add_suffix('_copy_tsne')
copy_nmf_df = copy_nmf_df.add_suffix('_copy_nmf')

In [7]:
all_data_combined = (rna_tsne_df.merge(rna_nmf_df, how = 'inner', left_index=True, right_index=True)
                     .merge(rna_pca_df, how = 'inner', left_index=True, right_index=True)
                     .merge(rna_vae_df, how = 'inner', left_index=True, right_index=True)
                     .merge(copy_pca_df, how = 'inner', left_index=True, right_index=True)
                     .merge(copy_tsne_df, how = 'inner', left_index=True, right_index=True)
                     .merge(copy_nmf_df, how = 'inner', left_index=True, right_index=True)
                     .merge(real_clinical_df, how = 'inner', left_index=True, right_index=True))

In [8]:
all_data_combined.to_csv('data_feature_encodings.tsv', sep='\t')